In [1]:
from textwrap import dedent

def parse_facets(s):
    return tuple(
        tuple(map(int, line.strip().split()))
        for line in s.splitlines()
    )

In [2]:
def perm_cycles(a, b):
    todo = set(a)

    while len(todo) > 0:
        do = min(todo)

        cycle = [do]
        while do in todo:
            todo.remove(do)
            do_i = next( i for i, val in enumerate(a) if val == do )
            don = b[do_i]
            if don == do or don == cycle[0]: break

            cycle.append(don)
            do = don
        if len(cycle) > 1:
            yield tuple(cycle)

def perm_string(a, b):
    return ' '.join(f'({" ".join(map(str, c))})' for c in perm_cycles(a, b))

In [3]:
def find_cycle(seq, eq=lambda a, b: a == b):
    tortoise = 1
    hare = 2
    while not eq(seq[tortoise], seq[hare]):
        tortoise += 1
        hare += 2

    mu = 0
    tortoise = 0
    while not eq(seq[tortoise], seq[hare]):
        tortoise += 1
        hare += 1
        mu += 1

    lam = 1
    hare = tortoise + 1
    while not eq(seq[tortoise], seq[hare]):
        hare += 1
        lam += 1

    return mu + lam

In [4]:
def facet_sums(perm, facets):
    return tuple(sum(perm[n-1] for n in f) for f in facets)

def facet_sum_dev(perm, facets):
    sums = tuple(sum(perm[n-1] for n in f) for f in facets)
    lo = min(sums)
    hi = max(sums)
    w = hi - lo
    h = lo + w/2
    return sum((s - h)/w for s in sums)

def facet_sum_var(perm, facets):
    sums = tuple(sum(perm[n-1] for n in f) for f in facets)
    lo = min(sums)
    hi = max(sums)
    return hi - lo

In [5]:
primary_facets = parse_facets(dedent('''\
     1  2  3  4
     5  6  7  8
     9 10 11 12
    13 14 15 16
    17 18 19 20
    21 22 23 24
'''))

secondary_facets = parse_facets(dedent('''\
    1 2 15
    1 15 16
    1 16 22
    1 22 4
    2 3 20
    2 20 17
    2 17 15
    3 4 5
    3 5 6
    3 6 20
    4 22 23
    4 23 5
    5 23 8
    6 7 19
    6 19 20
    7 8 9
    7 9 10
    7 10 19
    8 23 24
    8 24 9
    9 24 12
    10 11 18
    10 18 19
    11 12 13
    11 13 14
    11 14 18
    12 24 21
    12 21 13
    13 21 16
    14 15 17
    14 17 18
    16 21 22
'''))

In [6]:
from collections import namedtuple

Move = namedtuple('Move', ['score', 'perm', 'i', 'j', 'reason'])

def move_eq(a, b):
    i, j = a.i, a.j
    k, l = b.i, b.j
    if j < i: i, j = j, i
    if l < k: k, l = l, k
    return i == k and j == l

def apply_moves(perm, moves):
    ar = list(perm)
    for move in moves:
        i, j = move.i, move.j
        ar[i], ar[j] = ar[j], ar[i]
    return tuple(ar)

def ichoice(seq):
    for i in range(len(seq)):
        yield seq[i], seq[:i] + seq[i+1:]

In [7]:
from itertools import product
import math

def facet_sum_improvements(perm, facets, aspect):
    sums = tuple(sum(perm[n-1] for n in f) for f in facets)
    by_sum = sorted(enumerate(sums), key=lambda iv: iv[1])
    
    # min/max facet cursors in sorted-by-sum order
    i, j = 0, len(sums)-1
   
    while True:
        diff = by_sum[j][1] - by_sum[i][1]
        if diff <= 0: break

        fi = by_sum[i][0]
        fj = by_sum[j][0]
        for (to_swap, to_keep), (from_swap, from_keep) in product(ichoice(facets[fi]), ichoice(facets[fj])):
            if to_swap == from_swap: continue

            to_rem = sum(perm[n-1] for n in to_keep)
            from_rem = sum(perm[n-1] for n in from_keep)

            to_val = perm[to_swap-1]
            from_val = perm[from_swap-1]

            delta = from_val - to_val
            if delta <= 0: continue

            a_sum = to_rem + from_val
            b_sum = from_rem + to_val
            new_var = b_sum - a_sum
            score = math.inf if new_var == 0 else delta / new_var

            yield Move(score, perm, to_swap-1, from_swap-1, f'{aspect} sum balance d:{delta} a:{facets[fi]} b:{facets[fj]}')
    
        break # TODO wen i++ wen j--

In [8]:
def optimize(perm, mover, limit=10000):
    ar = list(perm)
    moves = []

    for _ in range(limit):
        for move in mover(ar):
            i, j = move.i, move.j
            ar[i], ar[j] = ar[j], ar[i]
            moves.append(move)
            break
        else: break

        try:
            end = find_cycle(moves, eq=move_eq)
        except IndexError:
            continue
        else:
            moves = moves[:end]
            break

    return moves

In [9]:
history = []
ar = tuple(range(1, 25))
history.append(ar)

movers = [
    lambda perm: facet_sum_improvements(perm, secondary_facets, '3-facet'),
    lambda perm: facet_sum_improvements(perm, primary_facets, '4-facet'),
]

for i in range(100):
    mover = movers[i % len(movers)]

    moves = optimize(ar, mover)
    #for move in moves: print(move)
    new = apply_moves(ar, moves)
    ar = new
    history.append(ar)
    
    try:
        end = find_cycle(history)
    except IndexError: pass
    else:
        history = history[:end]
        break

prior = history[0]
for perm in history[1:]:
    print(perm_string(prior, perm))
    prior = perm

print('===')

print(perm_string(history[0], history[-1]))

(1 4 23 8 7 14 5 12) (3 16)
(4 9 24 20 18 6 12 21) (5 10 13 22 19 17)
(1 23) (2 10 16 14 13 24) (3 11)
(5 21) (9 22)
(1 13) (3 16 5 22 23 9 24 14)
(2 12 21 24)
(1 2 23 21 9 12)
(1 24 9 22)
(1 22 2 3 16 5 24) (14 23)
(1 21) (2 14) (22 24)
(1 3 21) (2 16)
(1 3 24 16)
(1 21 24)
(1 3 10 16 23 21)
(1 24 2 14 5)
(5 23 10 24)
(1 5 24) (2 10 23)
(1 2 23 24)
(1 14) (3 23 10)
(2 14) (10 23 24)
(1 16) (2 23)
(2 23) (10 24)
(1 2 16)
(2 22 16 10 24)
(2 24 23 22 3)
(3 23 24)
(1 24) (3 23)
(1 23)
(1 24 23) (3 22)
(1 24) (3 22)
(1 23 24) (3 22)
===
(2 22 19 17 9 14 16 11 5 23 8 7) (3 24 20 18 6) (4 13 10 21)


In [10]:
v1 = [
    facet_sum_var(perm, primary_facets)
    for perm in history
]

In [11]:
v2 = [
    facet_sum_var(perm, secondary_facets)
    for perm in history
]

In [12]:
[x * y for x,y in zip(v1, v2)]

[3760,
 1408,
 312,
 960,
 1000,
 945,
 1312,
 1075,
 1394,
 1260,
 777,
 1254,
 675,
 1634,
 1564,
 1085,
 640,
 1089,
 1080,
 946,
 451,
 1666,
 782,
 1406,
 1125,
 525,
 1215,
 650,
 1488,
 943,
 1457,
 1050]

In [13]:
scix = sorted(range(len(history)), key=lambda i: v1[i] * v2[i])

In [14]:
[
    (i, v1[i] * v2[i], v1[i], v2[i])
    for i in scix
]

[(2, 312, 8, 39),
 (20, 451, 11, 41),
 (25, 525, 15, 35),
 (16, 640, 16, 40),
 (27, 650, 13, 50),
 (12, 675, 15, 45),
 (10, 777, 21, 37),
 (22, 782, 17, 46),
 (29, 943, 23, 41),
 (5, 945, 21, 45),
 (19, 946, 22, 43),
 (3, 960, 30, 32),
 (4, 1000, 25, 40),
 (31, 1050, 25, 42),
 (7, 1075, 25, 43),
 (18, 1080, 24, 45),
 (15, 1085, 35, 31),
 (17, 1089, 33, 33),
 (24, 1125, 25, 45),
 (26, 1215, 27, 45),
 (11, 1254, 38, 33),
 (9, 1260, 30, 42),
 (6, 1312, 32, 41),
 (8, 1394, 34, 41),
 (23, 1406, 38, 37),
 (1, 1408, 44, 32),
 (30, 1457, 31, 47),
 (28, 1488, 31, 48),
 (14, 1564, 34, 46),
 (13, 1634, 43, 38),
 (21, 1666, 49, 34),
 (0, 3760, 80, 47)]

In [15]:
history[scix[0]]

(9,
 2,
 16,
 23,
 21,
 12,
 14,
 7,
 24,
 13,
 11,
 1,
 22,
 10,
 15,
 3,
 5,
 6,
 17,
 18,
 4,
 19,
 8,
 20)